Alright — I’ll prepare **in-depth notes** for *DBMS – Concurrency Control/1* based fully on the lecture transcript you provided.
I’ll structure them so they cover **every single topic mentioned**, aligned with the stated learning outcomes, and keep them clear, logical, and detailed enough for revision or exam preparation.

---

# **DBMS – Concurrency Control/1**

## **Learning Outcomes**

1. Understand why designing serializable schedules in advance is difficult.
2. Learn about **Lock-Based Protocols** for concurrency control.
3. Understand how **locks** can be implemented and used to ensure **conflict serializability** and **recoverability**.
4. Recognize issues such as **deadlock**, **starvation**, and trade-offs between safety and concurrency.

---

## 1. **Background**

Before this module:

* We studied **serializability** (conflict & view), **recoverability**, and **cascade/cascadeless rollback**.
* View serializability is weaker than conflict serializability but checking it is **NP-complete** (expensive to test).
* Using only serializability & recoverability checks **after** execution is too late to prevent issues.

---

## 2. **Need for Concurrency Control**

* Schedules may still not be **adequately concurrent** even if they’re serializable and recoverable.
* We need **runtime mechanisms** to:

  * Allow high concurrency.
  * Maintain **ACID** properties.
  * Ensure recoverability.
* Naive approach: let only **one transaction run at a time** → safe but poor concurrency & throughput.
* **Concurrency control schemes** balance:

  * Concurrency level.
  * Overhead in managing locks/checks.

---

## 3. **Goal**

* Develop **protocols** that ensure serializability **dynamically**.
* Must prevent conflicts *before* they happen, instead of detecting them afterward.
* **Main approach:** Isolation through **mutual exclusion** on data items → Lock-Based Protocol.

---

## 4. **Lock-Based Protocols**

### 4.1 Idea

* Ensure **mutually exclusive** access to a data item.
* Instead of locking the whole database (serial execution), **lock specific data items**.

### 4.2 Types of Locks

1. **Exclusive Lock (X-lock)**:

   * Allows **read + write**.
   * Only **one transaction** can hold X-lock on a data item.
   * Requested via `lock-X(data_item)`.

2. **Shared Lock (S-lock)**:

   * Allows **read only**.
   * Multiple transactions can hold S-lock on the same item simultaneously.
   * Requested via `lock-S(data_item)`.

---

### 4.3 Lock Compatibility Matrix

| Request → / Held ↓ | **S-lock** | **X-lock** |
| ------------------ | ---------- | ---------- |
| **S-lock**         | Yes        | No         |
| **X-lock**         | No         | No         |

* **Yes**: Lock can be granted concurrently.
* **No**: Lock cannot be granted until the other is released.

---

### 4.4 Lock Protocol Rules

1. Transaction requests lock from **Lock Manager**.
2. If compatible → lock is granted.
3. If not compatible → transaction waits.
4. **Hold** the lock as long as you need the data.
5. **Unlock** when done (but unlocking too early may cause inconsistency).

---

## 5. **Example: Unlocking Too Early Problem**

Scenario:

* **T1** transfers \$50 from B → A.
* **T2** prints A + B.

If T1:

* Locks B → updates → unlocks B (before updating A).
* T2 can then read the **new B** but the **old A** → inconsistent sum.
* Output may be **250** instead of correct **300**.

**Fix:** Delay unlocking B until **after** updating A → prevents T2 from reading inconsistent state.

---

## 6. **Deadlock Problem**

**Deadlock example:**

* **T3** holds X-lock on B, needs X-lock on A.
* **T4** holds S-lock on A, needs S-lock on B.
* Both wait for each other → **deadlock**.

**Resolution:**

* Roll back one transaction → release locks → break cycle.

**Key point:**
Deadlock is **possible in any locking protocol**.

---

## 7. **Why Prefer Lock-Based Protocols Despite Deadlocks**

* Without locks → inconsistent states possible → **real-world damage**.
* With locks:

  * Risk of deadlocks → can be handled via rollback.
  * Inconsistency prevention is worth the trade-off.

---

## 8. **Two-Phase Locking (2PL) Protocol**

**Definition:**
A schedule is **conflict-serializable** if **all transactions** follow **Two-Phase Locking**.

**Phases:**

1. **Growing Phase**:

   * May acquire locks.
   * Cannot release any locks.
2. **Shrinking Phase**:

   * May release locks.
   * Cannot acquire any locks.

**Properties:**

* Ensures **conflict serializability**.
* Not all conflict-serializable schedules can be generated by 2PL.

---

### 8.1 Lock Conversion in 2PL

* **Growing Phase**:

  * Acquire new locks.
  * Upgrade: S-lock → X-lock.
* **Shrinking Phase**:

  * Release locks.
  * Downgrade: X-lock → S-lock.

---

## 9. **Automatic Lock Acquisition**

Systems can acquire locks **automatically**:

* **Read**: Acquire S-lock if no X-lock exists.
* **Write**: Acquire X-lock if no other locks exist.

  * If only this transaction has S-lock → upgrade to X-lock.

---

## 10. **Limitations of 2PL**

* **Deadlock** still possible.
* **Starvation (Livelock)**:

  * A transaction waits too long for a lock.
  * Others repeatedly get the lock first.

---

## 11. **Strict & Rigorous 2PL**

* **Strict 2PL**:

  * Hold **all X-locks** until commit/abort.
  * Prevents **cascading rollback**.
* **Rigorous 2PL**:

  * Hold **all locks** (S and X) until commit/abort.
  * Transactions serialized in order of commit.

---

## 12. **Trade-Off**

* **Stricter locking** → safer execution (less rollback) but **less concurrency**.
* **Looser locking** → more concurrency but higher risk of cascading rollback/inconsistency.

---

## 13. **Lock Implementation**

* Managed by **Lock Manager**.
* **Lock Table**: For each data item:

  * Transactions holding locks.
  * Lock type (S or X).
  * Transactions waiting for locks.
* Data structures:

  * Linked lists for granted & waiting locks.
  * References to locks held by a transaction (for rollback).

---

## 14. **Summary**

* Designing serializable schedules ahead of time is impractical → use runtime locking protocols.
* Locks provide mutual exclusion → maintain isolation.
* Two-phase locking ensures conflict serializability but not deadlock freedom.
* Variants like strict & rigorous 2PL improve recoverability.
* Trade-off between concurrency and safety.
* Lock manager + lock table are essential for implementation.